In [1]:
!pip install pyspark requests

from pyspark.sql import SparkSession
from pyspark.sql.functions import count
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=7024419b06178f713ba8234ed87e76aebc2269cac11227ca55b5c4331e0e37cd
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark
Mounted at /content/drive


In [2]:
spark = SparkSession.builder.appName("Gold").getOrCreate()

In [3]:
# Leitura dos dados no Drive
df_silver = spark.read.parquet('/content/drive/My Drive/silver.parquet')
df_silver.show(truncate=False)

+------------------------------------+---------------------------------------------+------------+----------------+--------------+----------------------------+
|Id                                  |Name                                         |Brewery_type|City            |State         |Brewery_Location            |
+------------------------------------+---------------------------------------------+------------+----------------+--------------+----------------------------+
|0faa0fb2-fffa-416d-9eab-46f67477c8ef|12 West Brewing Company - Production Facility|micro       |Mesa            |Arizona       |Mesa, Arizona               |
|4788221a-a03b-458c-9084-4cadd69ade6d|14 Cannons Brewing Company                   |micro       |Westlake Village|California    |Westlake Village, California|
|232e8f62-9afc-45f5-b4bc-582c26b5c43b|10th District Brewing Company                |micro       |Abington        |Massachusetts |Abington, Massachusetts     |
|9f1852da-c312-42da-9a31-097bac81c4c0|10 Barre

In [4]:
# Agrupamento fazendo uma contagem de quantas cervejarias existem em cada localidade
df_gold = df_silver.groupBy("Brewery_type", "Brewery_Location").agg(count("*").alias("Quantity"))
df_gold.show(50)

+------------+--------------------+--------+
|Brewery_type|    Brewery_Location|Quantity|
+------------+--------------------+--------+
|       micro|Mariposa, California|       1|
|       micro|Assumption, Illinois|       1|
|      closed|Bellingham, Washi...|       1|
|       micro|Saint Paul, Minne...|       1|
|       micro|Abington, Massach...|       1|
|       micro|       Mesa, Arizona|       1|
|     brewpub|Ocracoke, North C...|       1|
|  proprietor|    Denver, Colorado|       1|
|       micro|Milwaukee, Wisconsin|       1|
|       large|        Boise, Idaho|       1|
|     brewpub|Taylors, South Ca...|       1|
|       micro|Westlake Village,...|       1|
|       micro|    Killeshin, Laois|       1|
|       micro|        Reno, Nevada|       1|
|     brewpub|Williamsville, Ne...|       1|
|       micro|Castle Rock, Colo...|       1|
|    contract|Westminister, Mar...|       1|
|       large|        Bend, Oregon|       3|
|     brewpub|Holland Patent, N...|       1|
|       la

In [5]:
# Escrita no  Drive
df_gold.write.mode("overwrite").parquet("/content/drive/My Drive/gold.parquet")